In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [3]:
orders_df = pd.read_csv("orders.csv")
users_df = pd.read_json("users.json")

In [4]:
orders_df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian
...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian


In [5]:
users_df

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
...,...,...,...,...
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular


In [6]:
import pandas as pd
import sqlite3

orders_df = pd.read_csv("orders.csv")
users_df = pd.read_json("users.json")

conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

merged1 = pd.merge(
    orders_df,
    users_df,
    on="user_id",
    how="left"
)

final_df = pd.merge(
    merged1,
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.to_csv("final_food_delivery_dataset.csv", index=False)

print("✅ Final dataset created: final_food_delivery_dataset.csv")
print(final_df.head())


✅ Final dataset created: final_food_delivery_dataset.csv
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restauran

In [7]:
aa=pd.read_csv("final_food_delivery_dataset.csv")

In [8]:
aa.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
# Which city has the highest total revenue (total_amount) from Gold members? 
result = (
    aa[aa["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

result


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [10]:
#Which cuisine has the highest average order value across all orders?

result = (
    aa.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

result


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [11]:
#How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
result = (
    aa.groupby("user_id")["total_amount"]
    .sum()
    .reset_index()
    .query("total_amount > 1000")["user_id"]
    .nunique()
)

result


2544

In [13]:
#Which restaurant rating range generated the highest total revenue?
result = (
    aa.assign(
        rating_range=pd.cut(
            aa["rating"],
            bins=[3.0, 3.5, 4.0, 4.5, 5.0],
            labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"],
            include_lowest=True
        )
    )
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

result


C:\Users\ATTADEEP\AppData\Local\Temp\ipykernel_28640\1568587801.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [14]:
#Among Gold members, which city has the highest average order value?
result = (
    aa[aa["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

result


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [15]:
#Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
result = (
    aa.groupby("cuisine")
    .agg(
        distinct_restaurants=("restaurant_id", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .sort_values(
        by=["distinct_restaurants", "total_revenue"],
        ascending=[True, False]
    )
)

result


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [16]:
#What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
result = round(
    (aa[aa["membership"] == "Gold"].shape[0] / aa.shape[0]) * 100
)

result


50

In [17]:
#Which restaurant has the highest average order value but less than 20 total orders?
result = (
    aa[aa["restaurant_name_x"].isin([
        "Grand Cafe Punjabi",
        "Grand Restaurant South Indian",
        "Ruchi Mess Multicuisine",
        "Ruchi Foods Chinese"
    ])]
    .groupby("restaurant_name_x")
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "count")
    )
    .query("total_orders < 20")
    .sort_values("avg_order_value", ascending=False)
    .head(1)
)

result

,avg_order_value,total_orders
restaurant_name_x,,
Ruchi Foods Chinese,686.603158,19


In [19]:
#Which combination contributes the highest revenue?

result = (
    aa[aa.apply(
        lambda x: (
            (x["membership"] == "Gold" and x["cuisine"] == "Indian") or
            (x["membership"] == "Gold" and x["cuisine"] == "Italian") or
            (x["membership"] == "Regular" and x["cuisine"] == "Indian") or
            (x["membership"] == "Regular" and x["cuisine"] == "Chinese")
        ),
        axis=1
    )]
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

result


membership  cuisine
Gold        Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [20]:
#During which quarter of the year is the total revenue highest?
result = (
    aa.assign(order_date=pd.to_datetime(aa["order_date"], dayfirst=True))
    .assign(
        quarter=lambda x: x["order_date"].dt.month.map(
            lambda m: (
                "Q1 (Jan–Mar)" if m in [1, 2, 3] else
                "Q2 (Apr–Jun)" if m in [4, 5, 6] else
                "Q3 (Jul–Sep)" if m in [7, 8, 9] else
                "Q4 (Oct–Dec)"
            )
        )
    )
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

result




quarter
Q3 (Jul–Sep)    2037385.10
Q4 (Oct–Dec)    2018263.66
Q1 (Jan–Mar)    2010626.64
Q2 (Apr–Jun)    1945348.72
Name: total_amount, dtype: float64

In [21]:
#How many total orders were placed by users with Gold membership?
aa[aa["membership"] == "Gold"].shape[0]


4987

In [22]:
#What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
round(
    aa[aa["city"] == "Hyderabad"]["total_amount"].sum()
)


1889367

In [23]:
#How many distinct users placed at least one order?
aa["user_id"].nunique()



2883

In [24]:
#What is the average order value (rounded to 2 decimals) for Gold members?
int(round(
    aa[aa["membership"] == "Gold"]["total_amount"].mean(),
    2
))


797

In [25]:
#How many orders were placed for restaurants with rating ≥ 4.5?
aa[aa["rating"] >= 4].shape[0]

5594

In [26]:
#How many orders were placed in the top revenue city among Gold members only?
top_city = (
    aa[aa["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

aa[
    (aa["membership"] == "Gold") &
    (aa["city"] == top_city)
].shape[0]


1337

In [28]:
#The total number of rows in the final merged dataset is
aa.shape[0]


10000